In [1]:
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
device = "mps" if torch.mps.is_available() else "cpu"
device

'mps'